In [15]:
import numpy as np
import pandas as pd

# plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import matplotlib.pyplot as plt

# stats
from statsmodels.api import tsa # time series analysis
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX


from prophet import Prophet

In [16]:
file_path = '/Users/yingzhou/Downloads/Capstone_Babyname/Notebook_data/data.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows to verify
print(df.head())

   Unnamed: 0     Name  Year Gender  Count  Name_Ratio  Gender_Name_Ratio
0           0    Emily  2000      F  25959    6.867840          14.295257
1           1   Hannah  2000      F  23086    6.107745          12.713136
2           2  Madison  2000      F  19968    5.282832          10.996097
3           3   Ashley  2000      F  17998    4.761639           9.911246
4           4    Sarah  2000      F  17713    4.686238           9.754300


In [17]:
file_path = '/Users/yingzhou/Downloads/Capstone_Babyname/Notebook_data/dataset3.csv'

# Read the CSV file into a DataFrame
rank = pd.read_csv(file_path)

# Display the first few rows to verify
print(rank.head())


   Unnamed: 0  Rank      Name  Number Gender  Year
0           0     1    OLIVER     555   Male  2023
1           1     2      NOAH     537   Male  2023
2           2     3       LEO     409   Male  2023
3           3     4     HENRY     404   Male  2023
4           4     5  THEODORE     400   Male  2023


In [18]:
# Prove the prediction
# Filter for the year 2023
years_of_interest = [2023]
rank_filtered = rank[rank['Year'].isin(years_of_interest)]

# Top 100 male names for 2023
top_100_male_2023 = rank_filtered[(rank_filtered['Year'] == 2023) & (rank_filtered['Gender'] == 'Male')].head(100)

# Top 100 female names for 2023
top_100_female_2023 = rank_filtered[(rank_filtered['Year'] == 2023) & (rank_filtered['Gender'] == 'Female')].head(100)

# Display the results
print("Top 100 Male Names for 2023:")
print(top_100_male_2023)

print("\nTop 100 Female Names for 2023:")
print(top_100_female_2023)


Top 100 Male Names for 2023:
    Unnamed: 0  Rank      Name  Number Gender  Year
0            0     1    OLIVER     555   Male  2023
1            1     2      NOAH     537   Male  2023
2            2     3       LEO     409   Male  2023
3            3     4     HENRY     404   Male  2023
4            4     5  THEODORE     400   Male  2023
..         ...   ...       ...     ...    ...   ...
95          95    93    JOSHUA      89   Male  2023
96          96    97      ZAYN      88   Male  2023
97          97    97    MARCUS      88   Male  2023
98          98    97     ASHER      88   Male  2023
99          99   100       ALI      87   Male  2023

[100 rows x 6 columns]

Top 100 Female Names for 2023:
     Unnamed: 0  Rank       Name  Number  Gender  Year
100         100     1       ISLA     403  Female  2023
101         101     2     AMELIA     399  Female  2023
102         102     3     OLIVIA     381  Female  2023
103         103     4        MIA     347  Female  2023
104         104 

### Step 1: Filter the Top Names by Gender

In [34]:
# Filter the DataFrame for the year 2022
df_2022 = df[df['Year'] == 2022]

# Filter for females and sort by Count to get the top 100
top_100_female_2022 = df_2022[df_2022['Gender'] == 'F'].sort_values(by='Gender_Name_Ratio', ascending=False).head(100)

# Filter for males and sort by Count to get the top 100
top_100_male_2022 = df_2022[df_2022['Gender'] == 'M'].sort_values(by='Gender_Name_Ratio', ascending=False).head(100)

# Display the results
print("Top 100 Female Names by Gender_Name_Ratio for 2022:")
print(top_100_female_2022)

print("\nTop 100 Male Names by Gender_Name_Ratio for 2022:")
print(top_100_male_2022)

Top 100 Female Names by Gender_Name_Ratio for 2022:
         Unnamed: 0       Name  Year Gender  Count  Name_Ratio  \
1832354     1832354     Olivia  2022      F  16630    4.922141   
1832355     1832355       Emma  2022      F  14484    4.286969   
1832356     1832356  Charlotte  2022      F  12936    3.828792   
1832357     1832357     Amelia  2022      F  12400    3.670147   
1832358     1832358     Sophia  2022      F  12361    3.658604   
...             ...        ...   ...    ...    ...         ...   
1832449     1832449   Serenity  2022      F   2796    0.827559   
1832450     1832450    Liliana  2022      F   2784    0.824007   
1832451     1832451       Ayla  2022      F   2751    0.814240   
1832452     1832452    Raelynn  2022      F   2719    0.804769   
1832453     1832453  Everleigh  2022      F   2716    0.803881   

         Gender_Name_Ratio  
1832354          10.160956  
1832355           8.849747  
1832356           7.903916  
1832357           7.576419  
1832358   

### Step 2: Prepare Data for Prophet

In [38]:
def prepare_data_for_prophet(df_filtered):
    """Prepare data for Prophet"""
    # Validate input DataFrame
    if df_filtered.empty:
        raise ValueError("Filtered DataFrame is empty for the given name.")
    
    # Check required columns exist
    required_columns = {'Year', 'Gender_Name_Ratio'}
    if not required_columns.issubset(df_filtered.columns):
        raise ValueError("Filtered DataFrame is missing required columns.")
    
    # Check data types
    if not pd.api.types.is_numeric_dtype(df_filtered['Gender_Name_Ratio']):
        raise TypeError("Gender_Name_Ratio should be numeric.")
    
    df_prophet = df_filtered[['Year', 'Gender_Name_Ratio']].rename(columns={'Year': 'ds', 'Gender_Name_Ratio': 'y'})
    
    # Validate the transformed DataFrame
    if df_prophet.empty or df_prophet['ds'].isnull().any() or df_prophet['y'].isnull().any():
        raise ValueError("Prepared DataFrame has null values.")
    
    return df_prophet

# Validate the preparation for female names
for name, data in top_female_prophet_data.items():
    try:
        prepared_data = prepare_data_for_prophet(data)
        print(f"Data prepared successfully for {name}")
    except (ValueError, TypeError) as e:
        print(f"Validation error for {name}: {e}")

# Validate the preparation for male names
for name, data in top_male_prophet_data.items():
    try:
        prepared_data = prepare_data_for_prophet(data)
        print(f"Data prepared successfully for {name}")
    except (ValueError, TypeError) as e:
        print(f"Validation error for {name}: {e}")


Validation error for Olivia: Filtered DataFrame is missing required columns.
Validation error for Emma: Filtered DataFrame is missing required columns.
Validation error for Charlotte: Filtered DataFrame is missing required columns.
Validation error for Amelia: Filtered DataFrame is missing required columns.
Validation error for Sophia: Filtered DataFrame is missing required columns.
Validation error for Isabella: Filtered DataFrame is missing required columns.
Validation error for Ava: Filtered DataFrame is missing required columns.
Validation error for Mia: Filtered DataFrame is missing required columns.
Validation error for Evelyn: Filtered DataFrame is missing required columns.
Validation error for Luna: Filtered DataFrame is missing required columns.
Validation error for Harper: Filtered DataFrame is missing required columns.
Validation error for Camila: Filtered DataFrame is missing required columns.
Validation error for Sofia: Filtered DataFrame is missing required columns.
Valid

### Step 3: Forecast Using Prophet

In [39]:
def forecast_with_prophet(df_prophet):
    """Forecast using Prophet"""
    # Validate input DataFrame
    if df_prophet.empty:
        raise ValueError("Input DataFrame is empty.")
    
    model = Prophet()
    model.fit(df_prophet)
    
    # Make a future dataframe for 2023
    future = model.make_future_dataframe(periods=1, freq='Y')
    
    # Validate future DataFrame
    if future.empty:
        raise ValueError("Future DataFrame is empty.")
    
    forecast = model.predict(future)
    
    # Validate forecast DataFrame
    if forecast.empty or 'yhat' not in forecast.columns:
        raise ValueError("Forecast DataFrame is missing 'yhat' column.")
    
    # Extract the prediction for 2023
    forecast_2023 = forecast[forecast['ds'].dt.year == 2023]
    if forecast_2023.empty:
        raise ValueError("No forecast available for the year 2023.")
    
    prediction_2023 = forecast_2023['yhat'].values[0]
    
    return prediction_2023

# Validate forecasting for female names
for name, data in top_female_prophet_data.items():
    try:
        prediction = forecast_with_prophet(data)
        print(f"Forecast for {name} successfully completed.")
    except ValueError as e:
        print(f"Forecasting error for {name}: {e}")

# Validate forecasting for male names
for name, data in top_male_prophet_data.items():
    try:
        prediction = forecast_with_prophet(data)
        print(f"Forecast for {name} successfully completed.")
    except ValueError as e:
        print(f"Forecasting error for {name}: {e}")


08:56:48 - cmdstanpy - INFO - Chain [1] start processing
08:56:48 - cmdstanpy - INFO - Chain [1] done processing
/Users/yingzhou/anaconda3/envs/timeseries3/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(
08:56:48 - cmdstanpy - INFO - Chain [1] start processing
08:56:48 - cmdstanpy - INFO - Chain [1] done processing
/Users/yingzhou/anaconda3/envs/timeseries3/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(
08:56:48 - cmdstanpy - INFO - Chain [1] start processing
08:56:48 - cmdstanpy - INFO - Chain [1] done processing
/Users/yingzhou/anaconda3/envs/timeseries3/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.d

Forecasting error for Olivia: No forecast available for the year 2023.
Forecasting error for Emma: No forecast available for the year 2023.
Forecasting error for Charlotte: No forecast available for the year 2023.
Forecasting error for Amelia: No forecast available for the year 2023.
Forecasting error for Sophia: No forecast available for the year 2023.


/Users/yingzhou/anaconda3/envs/timeseries3/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(
08:56:48 - cmdstanpy - INFO - Chain [1] start processing
08:56:48 - cmdstanpy - INFO - Chain [1] done processing
/Users/yingzhou/anaconda3/envs/timeseries3/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(
08:56:48 - cmdstanpy - INFO - Chain [1] start processing
08:56:48 - cmdstanpy - INFO - Chain [1] done processing
/Users/yingzhou/anaconda3/envs/timeseries3/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(
08:56:48 - cmdstanpy - INFO - Chain [1] start processing
08:56:48 - cmdstanpy - INFO - Chain [1] done 

Forecasting error for Isabella: No forecast available for the year 2023.
Forecasting error for Ava: No forecast available for the year 2023.
Forecasting error for Mia: No forecast available for the year 2023.
Forecasting error for Evelyn: No forecast available for the year 2023.
Forecasting error for Luna: No forecast available for the year 2023.


08:56:48 - cmdstanpy - INFO - Chain [1] done processing
/Users/yingzhou/anaconda3/envs/timeseries3/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(
08:56:48 - cmdstanpy - INFO - Chain [1] start processing
08:56:48 - cmdstanpy - INFO - Chain [1] done processing
/Users/yingzhou/anaconda3/envs/timeseries3/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(
08:56:49 - cmdstanpy - INFO - Chain [1] start processing
08:56:49 - cmdstanpy - INFO - Chain [1] done processing
/Users/yingzhou/anaconda3/envs/timeseries3/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(
08:56:49 - cmdstanpy - INFO - Chain [1] start 

...

Forecasting error for Jordan: No forecast available for the year 2023.
Forecasting error for Jeremiah: No forecast available for the year 2023.
Forecasting error for Parker: No forecast available for the year 2023.


/Users/yingzhou/anaconda3/envs/timeseries3/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(
08:56:59 - cmdstanpy - INFO - Chain [1] start processing
08:56:59 - cmdstanpy - INFO - Chain [1] done processing
/Users/yingzhou/anaconda3/envs/timeseries3/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(
08:56:59 - cmdstanpy - INFO - Chain [1] start processing
08:56:59 - cmdstanpy - INFO - Chain [1] done processing
/Users/yingzhou/anaconda3/envs/timeseries3/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(
08:56:59 - cmdstanpy - INFO - Chain [1] start processing
08:56:59 - cmdstanpy - INFO - Chain [1] done 

Forecasting error for Greyson: No forecast available for the year 2023.
Forecasting error for Rowan: No forecast available for the year 2023.
Forecasting error for Nicholas: No forecast available for the year 2023.
Forecasting error for Adam: No forecast available for the year 2023.
Forecasting error for Theo: No forecast available for the year 2023.


08:56:59 - cmdstanpy - INFO - Chain [1] done processing
/Users/yingzhou/anaconda3/envs/timeseries3/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(


Forecasting error for Xavier: No forecast available for the year 2023.


### Step 4: Rank Predictions

In [41]:
# Function to validate ranking
def validate_ranking(predictions_df):
    """Validate the ranking of predictions"""
    if predictions_df.empty:
        raise ValueError("Predictions DataFrame is empty.")
    
    if not all(col in predictions_df.columns for col in ['Name', 'Predicted_Gender_Name_Ratio_2023']):
        raise ValueError("Predictions DataFrame is missing required columns.")
    
    if predictions_df['Predicted_Gender_Name_Ratio_2023'].isnull().any():
        raise ValueError("Predicted values contain nulls.")
    
    return predictions_df

# Validate ranking for female predictions
try:
    ranked_female_predictions = validate_ranking(predictions_female_2023)
    top_10_female_predictions_2023 = ranked_female_predictions.sort_values(by='Predicted_Gender_Name_Ratio_2023', ascending=False).head(10)
    print("Top 10 Female Predictions for 2023:")
    print(top_10_female_predictions_2023)
except ValueError as e:
    print(f"Ranking error: {e}")

# Validate ranking for male predictions
try:
    ranked_male_predictions = validate_ranking(predictions_male_2023)
    top_10_male_predictions_2023 = ranked_male_predictions.sort_values(by='Predicted_Gender_Name_Ratio_2023', ascending=False).head(10)
    print("Top 10 Male Predictions for 2023:")
    print(top_10_male_predictions_2023)
except ValueError as e:
    print(f"Ranking error: {e}")


Ranking error: Predictions DataFrame is empty.
Ranking error: Predictions DataFrame is empty.
